<p> This code was written to retrieve power usage data from a PostgreSQL table and create a time series model using Meta's Prophet API. Certain lines of code have been removed for privacy reasons, however the code's functionallity remains intact.<p>

In [ ]:
import os
from sqlalchemy import create_engine, text
from google.cloud.sql.connector import Connector
import google.auth
import pandas as pd
from prophet import Prophet


DB_NAME="removed"
GCP_CONNECTION_NAME="removed"
GOOGLE_APPLICATION_CREDENTIALS='removed'
DB_USER="removed"
DB_PASS="removed"

# Added new line
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

def get_gcp_connector():
    connector = Connector()
    gcp_connector = connector.connect(
        GCP_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME,

    )
    return gcp_connector


# create an engine to the postgresql server, with no reference to the database
google.auth.default()
engine = create_engine("postgresql+pg8000://", creator=get_gcp_connector)
print(engine)

# Test the connection by running a simple query
def test_connection():
    try:
        with engine.connect() as connection:
            # Run a simple query
            result = connection.execute(text("SELECT NOW()"))
            # Fetch and print the result
            for row in result:
                print(f"Connected successfully. Current database time: {row[0]}")
    except Exception as e:
        print(f"Connection failed: {e}")

# Call the test function
test_connection()

In [ ]:
def execute_sql_query(query): # modified to return a list
    try:
        with engine.connect() as connection:
            list = []
            result = connection.execute(text(query))
            # Fetch all results if it's a SELECT query
            if query.strip().upper().startswith("SELECT"):
                rows = result.fetchall()
                for row in rows:
                    list.append(row)
                return list
            else:
                print("Query executed successfully.")
    except Exception as e:
        print(f"Error occurred: {e}")

In [ ]:
def find_df_in_range(start_date: str, end_date: str) -> pd.DataFrame:
    sql_query = 'SELECT real_power, timestamp FROM "super_meter_1.1" WHERE "timestamp" BETWEEN ' + start_date + ' AND ' + end_date
    results = execute_sql_query(sql_query)
    real_power = [row[0] for row in results]
    timestamps = [pd.Timestamp(row[1]) for row in results]
    df = pd.DataFrame({"time": timestamps, "real_power": real_power})
    df['time'] = pd.to_datetime(df['time'], format='%Y/%m/%d %H:%M:%S.%f', errors='coerce')
    df['time'] = df['time'].dt.floor('s')
    df = df.dropna(subset=['time'])
    df = df.rename(columns={"time": "ds", "real_power": "y"})
    return df

test = find_df_in_range("'2024-02-26 15:36:15'", "'2024-03-26 15:37:05'")

In [ ]:
m = Prophet()
m.fit(test)

In [ ]:
future = m.make_future_dataframe(periods=2)
future.tail()
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)